Nous allons détailler les fonctions dans ce notebook.

# IMPORTS , FONCTIONS ET URL

In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options # en vue de faire tourner le browser en headless
from pathlib import Path
import os
# from webdriver_manager.chrome import ChromeDriverManager
# import chromedriver_binary
import re
import csv
import time
regex_expression = "</span> #.* in Kindle Store " # utile pour regex
#DRIVER_PATH_chromedriver = str(Path('../chromedriver').resolve()) ancienne ligne qui fonctionnait

from amazoncaptcha import AmazonCaptcha #librairi pour passer l'image capthca d'amazon
from selenium.webdriver.common.by import By




In [31]:
urlcaptcha = "https://www.amazon.com/errors/validateCaptcha"

#attention sur PC DELL UBUNTU le Wagon, bien penser à lancer X launch , et à désactiver VPN !!

In [2]:
configuration = { "wagon":"/usr/bin/chromedriver/chromedriver" , 
                 "raspberry":"/usr/bin/chromedriver" ,
                "anaconda-PS-stock":""}

In [3]:
environnement = input('Entrez la configuration matérielle')

Entrez la configuration matériellewagon


In [4]:
chemin_access = configuration[environnement]

In [5]:
chemin_access

'/usr/bin/chromedriver/chromedriver'

In [6]:
DRIVER_PATH_chromedriver = str(Path(chemin_access).resolve()) # installation chrome driver sur usr/bin


chrome_options = Options()
chrome_options.add_argument("--headless")

browser = webdriver.Chrome(executable_path=DRIVER_PATH_chromedriver, options=chrome_options)

# browser = webdriver.Chrome(executable_path=DRIVER_PATH_chromedriver)

browser.get("http://www.amazon.com")

#browser.get("http://www.amazon.com") #ici chrome est norma
#browser.get("http://www.amazon.com" , options = chome_options) # avec cette option le chrome est headless

/tmp/ipykernel_3693/2879895388.py:7: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome(executable_path=DRIVER_PATH_chromedriver, options=chrome_options)


In [7]:
browser = webdriver.Chrome(executable_path=DRIVER_PATH_chromedriver)
browser.get("https://www.amazon.com/errors/validateCaptcha")

/tmp/ipykernel_3693/2394049935.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome(executable_path=DRIVER_PATH_chromedriver)


In [8]:
link = browser.find_element(By.XPATH,"//div[@class = 'a-row a-text-center']//img").get_attribute('src')
captcha = AmazonCaptcha.fromlink(link)
captcha_value = AmazonCaptcha.solve(captcha)
input_field = browser.find_element(By.ID, "captchacharacters").send_keys(captcha_value)
button = browser.find_element(By.CLASS_NAME,"a-button-text")
button.click()

In [50]:
def passer_le_captcha():
    link = browser.find_element(By.XPATH,"//div[@class = 'a-row a-text-center']//img").get_attribute('src')
    captcha = AmazonCaptcha.fromlink(link)
    captcha_value = AmazonCaptcha.solve(captcha)
    input_field = browser.find_element(By.ID, "captchacharacters").send_keys(captcha_value)
    button = browser.find_element(By.CLASS_NAME,"a-button-text")
    button.click()

In [11]:
captcha_value

'FMNBFE'

In [9]:
liste_categorie = pd.read_csv('liste_categorie.txt' , sep ='\t', index_col = "Numero")

In [10]:
liste_categorie['url'][2]

'https://www.amazon.com/s?bbn=157632011&rh=n%3A133140011%2Cn%3A154606011%2Cn%3A154607011%2Cn%3A157630011%2Cn%3A157632011%2Cn%3A157635011&dc&qid=1689403032&rnid=157632011&ref=lp_157632011_nr_n_1'

In [27]:
liste_categorie_avec_Captcha = liste_categorie

In [28]:
liste_categorie_avec_Captcha['url'].iloc[1] = "urlcaptcha"

In [29]:
liste_categorie_avec_Captcha

,url,cat1,cat2,cat3,cat4,cat5,cat6
Numero,,,,,,,
0,https://www.amazon.com/s?bbn=157630011&rh=n%3A...,Arts & Photography,Architecture,"Architects, A-Z",NaN,NaN,NaN
1,urlcaptcha,Arts & Photography,Architecture,Building Types & Styles,Religious Buildings,NaN,NaN
2,https://www.amazon.com/s?bbn=157632011&rh=n%3A...,Arts & Photography,Architecture,Building Types & Styles,Specific Styles,NaN,NaN
3,https://www.amazon.com/s?bbn=157630011&rh=n%3A...,Arts & Photography,Architecture,Criticism,NaN,NaN,NaN
4,https://www.amazon.com/s?bbn=157630011&rh=n%3A...,Arts & Photography,Architecture,Drawing & Modelling,NaN,NaN,NaN
...,...,...,...,...,...,...,...
5133,https://www.amazon.com/s?bbn=160000011&rh=n%3A...,Travel,United States,States,Washington,NaN,NaN
5134,https://www.amazon.com/s?bbn=160000011&rh=n%3A...,Travel,United States,States,West Virginia,NaN,NaN
5135,https://www.amazon.com/s?bbn=160000011&rh=n%3A...,Travel,United States,States,Wisconsin,NaN,NaN


In [26]:
liste_categorie

,url,cat1,cat2,cat3,cat4,cat5,cat6
Numero,,,,,,,
0,https://www.amazon.com/s?bbn=157630011&rh=n%3A...,Arts & Photography,Architecture,"Architects, A-Z",NaN,NaN,NaN
1,https://www.amazon.com/s?bbn=157632011&rh=n%3A...,Arts & Photography,Architecture,Building Types & Styles,Religious Buildings,NaN,NaN
2,https://www.amazon.com/s?bbn=157632011&rh=n%3A...,Arts & Photography,Architecture,Building Types & Styles,Specific Styles,NaN,NaN
3,https://www.amazon.com/s?bbn=157630011&rh=n%3A...,Arts & Photography,Architecture,Criticism,NaN,NaN,NaN
4,https://www.amazon.com/s?bbn=157630011&rh=n%3A...,Arts & Photography,Architecture,Drawing & Modelling,NaN,NaN,NaN
...,...,...,...,...,...,...,...
5133,https://www.amazon.com/s?bbn=160000011&rh=n%3A...,Travel,United States,States,Washington,NaN,NaN
5134,https://www.amazon.com/s?bbn=160000011&rh=n%3A...,Travel,United States,States,West Virginia,NaN,NaN
5135,https://www.amazon.com/s?bbn=160000011&rh=n%3A...,Travel,United States,States,Wisconsin,NaN,NaN


In [16]:
liste_categorie.groupby(["cat1"]).count()

,url,cat2,cat3,cat4,cat5,cat6
cat1,,,,,,
Arts & Photography,198,198,195,119,31,0
Biographies & Memoirs,104,104,98,45,9,0
Business & Money,213,213,211,55,2,0
Children s eBooks,363,363,363,185,17,0
"Comics, Manga , Graphic Novel",1,0,0,0,0,0
Computers & Technology,75,75,66,3,0,0
"Cookbooks, Food & Wine",114,114,106,40,0,0
"Crafts, Hobbies & Home Kindle",191,191,190,41,0,0
Education & Teaching Kindle,81,81,81,54,4,0


# DECODER LES URL DES  DES CATEGORIES ET SOUS CATEGORIES

In [12]:
liste_categorie['url'][0]

'https://www.amazon.com/s?bbn=157630011&rh=n%3A133140011%2Cn%3A154606011%2Cn%3A154607011%2Cn%3A157630011%2Cn%3A157631011&dc&qid=1689403029&rnid=157630011&ref=lp_157630011_nr_n_0'

In [13]:
liste_categorie['url'][0].split('&dc')[0].split('%')[-1]

'3A157631011'

In [14]:
def trouver_node_url(url):
    """fonction qui retourne un lien simplifié et que l'on peu incrémenter d'une catégorie"""
    node = url.split('&dc')[0].split('%')[-1]
    lien = 'https://www.amazon.com/s?i=digital-text&rh=n%' + node +'&fs=true&page=1'
    #browser.get(lien)
    return(lien)

def trouver_node_url_a_incrementer(url):
    """fonction qui retourne un url que l'on peu incrémenter simplement il n'y a plus qu'a adjoindre i et l'incrémenter à la fin"""
    node = url.split('&dc')[0].split('%')[-1]
    lien = 'https://www.amazon.com/s?i=digital-text&rh=n%' + node +'&fs=true&page='
    #browser.get(lien)
    return(lien)

def y_a_t_il_des_livres_dans_page(url): #fonction qui détermine s'il y a des livres dans la page
    """ retourne True s'il y a des livres dans la page et False s'il n'y en a pas"""
    browser.get(url) #navigue sur une page avec tous les liens de la liste
    html_page_listing = browser.page_source # permet de prendre le code html de la page 
    soup = BeautifulSoup(html_page_listing , 'lxml') # créer la soupe avec le parser
    livres = soup.find_all('div' ,{ 'data-asin':True, 'data-component-type' : 's-search-result'} ) # trouve tous les livres de la pages
    if livres!=[]:
        return True
    else : 
        return False
    
    
def trouver_les_livres_dans_page(url_page_principale):
    """fonction qui retourne une liste de lien web des livres présents dans la page"""
    browser.get(url_page_principale)
    html_page_listing = browser.page_source # permet de prendre le code html de la page 
    soup = BeautifulSoup(html_page_listing , 'lxml') # créer la soupe avec le parser
    livres = soup.find_all('a',class_ = 'a-link-normal s-underline-text s-underline-link-text s-link-style a-text-normal')
    liens_livres=['https://www.amazon.com'+livre.get('href') for livre in livres]  
    #on utilise liste comprehension pour prendre l'élement href (lien hypertexte) 
    # et ajouter www.amazon.com/ pour avoir un url exploitable
    return(liens_livres)

def compter_nb_livre_dans_page(url_page_principale):
    """fonction qui retourne une liste de lien web des livres présents dans la page"""
    browser.get(url_page_principale)
    html_page_listing = browser.page_source # permet de prendre le code html de la page 
    soup = BeautifulSoup(html_page_listing , 'lxml') # créer la soupe avec le parser
    livres = soup.find_all('a',class_ = 'a-link-normal s-underline-text s-underline-link-text s-link-style a-text-normal')
    liens_livres=['https://www.amazon.com'+livre.get('href') for livre in livres]  
    #on utilise liste comprehension pour prendre l'élement href (lien hypertexte) 
    # et ajouter www.amazon.com/ pour avoir un url exploitable
    return(len(liens_livres))

def rang_livre(url_d_un_livre):
    """ fonction qui retour le rang des vente d'une livre """
    browser.get(url_d_un_livre)
    html_page_listing = browser.page_source # permet de prendre le code html de la page 
    soup = BeautifulSoup(html_page_listing , 'lxml') # créer la soupe avec le parser
    if soup.find_all( {'span'  : 'a-list-item'}) is not None : 
        rang_livre = soup.find_all( {'span'  : 'a-list-item'})
        if len(str(re.compile(regex_expression).search(str(rang_livre))).split())>5 : 
            rang = int(str(re.compile(regex_expression).search(str(rang_livre))).split()[5].replace("#","").replace(",",""))
        else : rang = None
    else : rang = None
    return(rang)


def titre_livre(url_d_un_livre):
    """fonction qui retourne le titre du livre lorsqu'on visite la page du livre """
    browser.get(url_d_un_livre)
    html_page_listing = browser.page_source # permet de prendre le code html de la page 
    soup = BeautifulSoup(html_page_listing , 'lxml') # créer la soupe avec le parser
#     if soup.find('span',class_ = "a-size-extra-large celwidget").text == None: 
#         pass
#     else: 
    titre=soup.find('span',class_ = "a-size-extra-large celwidget").text.strip()
        
    return(titre)


def lien_livre(url_d_un_livre):
    """fonction qui retour le lien du livre  """
    return(url_d_un_livre)
    


In [15]:
def scrapper_un_livre(url_d_un_livre):
    
    browser.get(url_d_un_livre)
    html_page_listing = browser.page_source # permet de prendre le code html de la page 
    soup = BeautifulSoup(html_page_listing , 'lxml') # créer la soupe avec le parser
    
    titre=soup.find('span',class_ = "a-size-extra-large celwidget").text.strip() #retour le titre du livre
    
    if soup.find_all( {'span'  : 'a-list-item'}) is not None : 
        rang_livre = soup.find_all( {'span'  : 'a-list-item'})
        if len(str(re.compile(regex_expression).search(str(rang_livre))).split())>5 : 
            rang = int(str(re.compile(regex_expression).search(str(rang_livre))).split()[5].replace("#","").replace(",",""))
        else : rang = None
    else : rang = None
    
    return(list((titre,rang,url_d_un_livre)))

    

In [16]:
scrapper_un_livre("https://www.amazon.com/Brunelleschis-Dome-Renaissance-Reinvented-Architecture-ebook/dp/B00DUVKOUW/ref=sr_1_1?qid=1707170304&s=digital-text&sr=1-1")

["Brunelleschi's Dome: How a Renaissance Genius Reinvented Architecture",
 133371,
 'https://www.amazon.com/Brunelleschis-Dome-Renaissance-Reinvented-Architecture-ebook/dp/B00DUVKOUW/ref=sr_1_1?qid=1707170304&s=digital-text&sr=1-1']

In [17]:
tableau_principal = []
for i in range(0,2): # i = 0 #doit aller de 0 à 5136 il y a 5137 éléments categoire
    ligne_categorie = liste_categorie.iloc[i].tolist()
    #prends un éléments du dataframe en liste
    # permet de convertir un élément du dataframe en liste
    for j in range(1,3):# doit aller de 1 à ce qu'il n'y ait plus de page mais on peut partir de 1 à 2 ou 3 au départ.
        page_livres_multi=trouver_node_url_a_incrementer(liste_categorie.iloc[i].tolist()[0]) + str(j)
        page_livres_multi
        print(page_livres_multi)

https://www.amazon.com/s?i=digital-text&rh=n%3A157631011&fs=true&page=1
https://www.amazon.com/s?i=digital-text&rh=n%3A157631011&fs=true&page=2
https://www.amazon.com/s?i=digital-text&rh=n%3A157634011&fs=true&page=1
https://www.amazon.com/s?i=digital-text&rh=n%3A157634011&fs=true&page=2


In [18]:
ligne_categorie = liste_categorie.iloc[1].tolist()
ligne_categorie[0]

'https://www.amazon.com/s?bbn=157632011&rh=n%3A133140011%2Cn%3A154606011%2Cn%3A154607011%2Cn%3A157630011%2Cn%3A157632011%2Cn%3A157634011&dc&qid=1689403032&rnid=157632011&ref=lp_157632011_nr_n_0'

In [19]:
page_livres_multi = trouver_node_url_a_incrementer(ligne_categorie[0])+str(1)
page_livres_multi

'https://www.amazon.com/s?i=digital-text&rh=n%3A157634011&fs=true&page=1'

In [20]:
trouver_les_livres_dans_page(page_livres_multi)

['https://www.amazon.com/Brunelleschis-Dome-Renaissance-Reinvented-Architecture-ebook/dp/B00DUVKOUW/ref=sr_1_1?qid=1707543110&s=digital-text&sr=1-1',
 'https://www.amazon.com/Universe-Stone-Chartres-Cathedral-Invention-ebook/dp/B001ANYCW8/ref=sr_1_2?qid=1707543110&s=digital-text&sr=1-2',
 'https://www.amazon.com/Shinto-Kami-Way-Sokyo-Ono-ebook/dp/B005M2AI0I/ref=sr_1_3?qid=1707543110&s=digital-text&sr=1-3',
 'https://www.amazon.com/Genius-Design-Bernini-Borromini-Transformed-ebook/dp/B00192MNO4/ref=sr_1_4?qid=1707543110&s=digital-text&sr=1-4',
 'https://www.amazon.com/Camino-Santiago-Travel-Guide-Pilgrims-ebook/dp/B0CRZ99QJ9/ref=sr_1_5?qid=1707543110&s=digital-text&sr=1-5',
 'https://www.amazon.com/Basilica-Splendor-Scandal-Building-Peters-ebook/dp/B000OZ0NNA/ref=sr_1_6?qid=1707543110&s=digital-text&sr=1-6',
 'https://www.amazon.com/Steeple-Chasing-Around-Britain-Church-ebook/dp/B0B1D41776/ref=sr_1_7?qid=1707543110&s=digital-text&sr=1-7',
 'https://www.amazon.com/Tomb-View-Stories-Grave

In [21]:
url="https://www.amazon.com/s?i=digital-text&rh=n%3A157631011&fs=true&page=2"

In [22]:
browser.get(url)

In [97]:
trouver_les_livres_dans_page("https://www.amazon.com/s?i=digital-text&rh=n%3A157634011&fs=true&qid=1707170086&ref=sr_pg_1")

['https://www.amazon.com/Brunelleschis-Dome-Renaissance-Reinvented-Architecture-ebook/dp/B00DUVKOUW/ref=sr_1_1?qid=1707173932&s=digital-text&sr=1-1',
 'https://www.amazon.com/Universe-Stone-Chartres-Cathedral-Invention-ebook/dp/B001ANYCW8/ref=sr_1_2?qid=1707173932&s=digital-text&sr=1-2',
 'https://www.amazon.com/Shinto-Kami-Way-Sokyo-Ono-ebook/dp/B005M2AI0I/ref=sr_1_3?qid=1707173932&s=digital-text&sr=1-3',
 'https://www.amazon.com/Genius-Design-Bernini-Borromini-Transformed-ebook/dp/B00192MNO4/ref=sr_1_4?qid=1707173932&s=digital-text&sr=1-4',
 'https://www.amazon.com/Basilica-Splendor-Scandal-Building-Peters-ebook/dp/B000OZ0NNA/ref=sr_1_5?qid=1707173932&s=digital-text&sr=1-5',
 'https://www.amazon.com/Tomb-View-Stories-Graveyards-Financial-ebook/dp/B094J23ZJ4/ref=sr_1_6?qid=1707173932&s=digital-text&sr=1-6',
 'https://www.amazon.com/Steeple-Chasing-Around-Britain-Church-ebook/dp/B0B1D41776/ref=sr_1_7?qid=1707173932&s=digital-text&sr=1-7',
 'https://www.amazon.com/Gothic-Enterprise-Unde

In [72]:
browser.get(urlcaptcha)
html_page_listing = browser.page_source # permet de prendre le code html de la page 
soup = BeautifulSoup(html_page_listing , 'lxml') # créer la soupe avec le parser
alerte = soup.find_all("p", class_="a-last")
if str(alerte) == """[<p class="a-last">Sorry, we just need to make sure you\'re not a robot. For best results, please make sure your browser is accepting cookies.</p>]""" :
    passer_le_captcha()
else : 
    sprint(soup)
# title = movie.find("p").find("a-last").string

In [34]:
soup

<html class="a-no-js" lang="en-us"><!--<![endif]--><head>
<meta content="text/html; charset=utf-8" http-equiv="content-type"/>
<meta charset="utf-8"/>
<meta content="IE=edge,chrome=1" http-equiv="X-UA-Compatible"/>
<title dir="ltr">Amazon.com</title>
<meta content="width=device-width" name="viewport"/>
<link href="https://images-na.ssl-images-amazon.com/images/G/01/AUIClients/AmazonUI-3c913031596ca78a3768f4e934b1cc02ce238101.secure.min._V1_.css" rel="stylesheet"/>
<script>

if (true === true) {
    var ue_t0 = (+ new Date()),
        ue_csm = window,
        ue = { t0: ue_t0, d: function() { return (+new Date() - ue_t0); } },
        ue_furl = "fls-na.amazon.com",
        ue_mid = "ATVPDKIKX0DER",
        ue_sid = (document.cookie.match(/session-id=([0-9-]+)/) || [])[1],
        ue_sn = "opfcaptcha.amazon.com",
        ue_id = '7P71T7W9B29VQAQP4S0G';
}
</script>
<script src="https://images-na.ssl-images-amazon.com/images/G/01/csminstrumentation/csm-captcha-instrumentation.min.js"></scr

In [24]:
browser.get(url_page_principale)
    html_page_listing = browser.page_source # permet de prendre le code html de la page 
    soup = BeautifulSoup(html_page_listing , 'lxml') # créer la soupe avec le parser
    livres = soup.find_all('a',class_ = 'a-link-normal s-underline-text s-underline-link-text s-link-style a-text-normal')
    liens_livres=['https://www.amazon.com'+livre.get('href') for livre in livres]  
    #on utilise liste comprehension pour prendre l'élement href (lien hypertexte) 
    # et ajouter www.amazon.com/ pour avoir un url exploitable
    return(liens_livres)

IndentationError: unexpected indent (2739723312.py, line 2)

In [25]:
tableau_principal = []
start = time.time()


for i in range(0,2): # i = 0 #doit aller de 0 à 5136 il y a 5137 éléments categoire
    ligne_categorie = liste_categorie.iloc[i].tolist()
    #prends un éléments du dataframe en liste
    # permet de convertir un élément du dataframe en liste
    for j in range(1,2):# doit aller de 1 à ce qu'il n'y ait plus de page mais on peut partir de 1 à 2 ou 3 au départ.
        page_livres_multi=trouver_node_url_a_incrementer(liste_categorie.iloc[i].tolist()[0]) + str(j)
        page_livres_multi
    # for livre in page_livres_multi:
    # trouver_les_livres_dans_page(page_livres_multi)  # faire un joli code utiliser que la fonction retourne une liste et balayer la liste
    
    # for k in range(1,10):
    #     trouver_les_livres_dans_page(page_livres_multi)[k]
    #     page_livre = trouver_les_livres_dans_page(page_livres_multi)[k]
    #     liste_livre = [titre_livre(page_livre) , rang_livre(page_livre), lien_livre(page_livre)]
    #     ligne_supp = ligne_categorie + liste_livre
    #     tableau_principal.append(ligne_supp)

        livres_a_scrapper = trouver_les_livres_dans_page(page_livres_multi)  # faire un joli code utiliser que la fonction retourne une liste et balayer la liste

        for livre in livres_a_scrapper:
            ligne_supp = ligne_categorie + scrapper_un_livre(livre)
            tableau_principal.append(ligne_supp)
end = time.time()
print(end - start)
tableau_principal

171.8694245815277


[['https://www.amazon.com/s?bbn=157630011&rh=n%3A133140011%2Cn%3A154606011%2Cn%3A154607011%2Cn%3A157630011%2Cn%3A157631011&dc&qid=1689403029&rnid=157630011&ref=lp_157630011_nr_n_0',
  'Arts & Photography',
  'Architecture',
  'Architects, A-Z',
  nan,
  nan,
  nan,
  'A Place of My Own: The Architecture of Daydreams',
  30984,
  'https://www.amazon.com/Place-My-Own-Architecture-Daydreams-ebook/dp/B00BE2568E/ref=sr_1_1?qid=1707543272&s=digital-text&sr=1-1'],
 ['https://www.amazon.com/s?bbn=157630011&rh=n%3A133140011%2Cn%3A154606011%2Cn%3A154607011%2Cn%3A157630011%2Cn%3A157631011&dc&qid=1689403029&rnid=157630011&ref=lp_157630011_nr_n_0',
  'Arts & Photography',
  'Architecture',
  'Architects, A-Z',
  nan,
  nan,
  nan,
  'The Stoic way of Life: The Ultimate Guide of Stoicism to make your Everyday Modern Life Calm, Confident - Master the Art of Living, Emotional Resilience & Perseverance (Mastering Stoicism)',
  49309,
  'https://www.amazon.com/Stoic-way-Life-Confident-Perseverance-ebook

In [84]:
titres_colonnes = ["url_categorie","cat1","cat2","cat3","cat4","cat5","cat6","titre","rang","lien"]

In [99]:
df = pd.DataFrame(tableau_principal, columns=titres_colonnes)

In [100]:
df

,url_categorie,cat1,cat2,cat3,cat4,cat5,cat6,titre,rang,lien
0,https://www.amazon.com/s?bbn=157630011&rh=n%3A...,Arts & Photography,Architecture,"Architects, A-Z",NaN,NaN,NaN,The Stoic way of Life: The Ultimate Guide of S...,58962,https://www.amazon.com/Stoic-way-Life-Confiden...
1,https://www.amazon.com/s?bbn=157630011&rh=n%3A...,Arts & Photography,Architecture,"Architects, A-Z",NaN,NaN,NaN,A Place of My Own: The Architecture of Daydreams,133880,https://www.amazon.com/Place-My-Own-Architectu...
2,https://www.amazon.com/s?bbn=157630011&rh=n%3A...,Arts & Photography,Architecture,"Architects, A-Z",NaN,NaN,NaN,Architectural Digest at 100: A Century of Style,25989,https://www.amazon.com/Architectural-Digest-10...
3,https://www.amazon.com/s?bbn=157630011&rh=n%3A...,Arts & Photography,Architecture,"Architects, A-Z",NaN,NaN,NaN,The $12 Million Stuffed Shark: The Curious Eco...,326804,https://www.amazon.com/Million-Stuffed-Shark-E...
4,https://www.amazon.com/s?bbn=157630011&rh=n%3A...,Arts & Photography,Architecture,"Architects, A-Z",NaN,NaN,NaN,Frank Lloyd Wright,741489,https://www.amazon.com/Frank-Lloyd-Wright-Mery...
5,https://www.amazon.com/s?bbn=157630011&rh=n%3A...,Arts & Photography,Architecture,"Architects, A-Z",NaN,NaN,NaN,The Gardner Heist: The True Story of the World...,565825,https://www.amazon.com/Gardner-Heist-Worlds-La...
6,https://www.amazon.com/s?bbn=157630011&rh=n%3A...,Arts & Photography,Architecture,"Architects, A-Z",NaN,NaN,NaN,Arana Orrego Torres (Spanish Edition),1255483,https://www.amazon.com/Orrego-Torres-Spanish-E...
7,https://www.amazon.com/s?bbn=157630011&rh=n%3A...,Arts & Photography,Architecture,"Architects, A-Z",NaN,NaN,NaN,Death in a Prairie House: Frank Lloyd Wright a...,678183,https://www.amazon.com/Death-Prairie-House-Tal...
8,https://www.amazon.com/s?bbn=157630011&rh=n%3A...,Arts & Photography,Architecture,"Architects, A-Z",NaN,NaN,NaN,The Fellowship: The Untold Story of Frank Lloy...,132548,https://www.amazon.com/Fellowship-Untold-Story...
9,https://www.amazon.com/s?bbn=157630011&rh=n%3A...,Arts & Photography,Architecture,"Architects, A-Z",NaN,NaN,NaN,Confessions Of an Art Addict,754818,https://www.amazon.com/Confessions-Art-Addict-...


In [76]:
df.to_csv('2024-02.csv')